In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_stones/GSE73680'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profile of Randall's Plaque tissue"
!Series_summary	"Randall’s plaque (RP) is the origin of renal calcification on which idiopathic calcium oxalate (CaOx) kidney stones develop. To establish genomic pathogenesis of RP, we performed the microarray analysis for comparing the gene expressions among renal papillary RP and normal tissue of 23 CaOx and 6 calcium phosphate (CaP) stone formers, and normal papillary tissue of 7 control patients. Compare to normal papillary tissue, RP tissue contained up-regulation of lipocalin 2, interleukin 11, prostaglandin-endoperoxide synthase 1, glutathione peroxidase 3, and monocyte to macrophage differentiation, whereas down-regulation of solute carrier family 12 member 1  and sodium leak channel non selective (either > 2.0- or 0.5-fold, p <0.01). The network and toxicity analysis showed these genes had association with activated mitogen-activated protein kinase, Akt/ phosphatidylinositol 3-kinase pa

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # Based on series summary, it performs microarray analysis for gene expressions

# Check availability for 'Kidney_stones' (variable inferred based on context from tissue type information)
trait_row = 2

# Check availability for 'age' (not explicitly mentioned in the characteristics)
age_row = None

# Check availability for 'gender' (explicitly mentioned)
gender_row = 0

def convert_trait(value):
    """Convert Kidney_stones trait data to binary"""
    val = value.split(":")[1].strip()
    if "control" in val:
        return 0
    else:
        return 1

def convert_age(value):
    """No available data, so return None"""
    return None

def convert_gender(value):
    """Convert gender data to binary"""
    val = value.split(":")[1].strip()
    if val == "female":
        return 0
    elif val == "male":
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE73680', './preprocessed/Kidney_stones/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_stones', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Kidney_stones/trait_data/GSE73680.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM1900673': [0, 0], 'GSM1900674': [0, 1], 'GSM1900675': [1, 0], 'GSM1900676': [1, 0], 'GSM1900677': [1, 1], 'GSM1900678': [1, 1], 'GSM1900679': [1, 1], 'GSM1900680': [1, 1], 'GSM1900681': [1, 1], 'GSM1900682': [1, 1], 'GSM1900683': [1, 1], 'GSM1900684': [0, 0], 'GSM1900685': [0, 1], 'GSM1900686': [1, 0], 'GSM1900687': [1, 0], 'GSM1900688': [1, 1], 'GSM1900689': [1, 1], 'GSM1900690': [1, 1], 'GSM1900691': [1, 1], 'GSM1900692': [1, 1], 'GSM1900693': [1, 1], 'GSM1900694': [1, 1], 'GSM1900695': [1, 1], 'GSM1900696': [1, 1], 'GSM1900697': [1, 1], 'GSM1900698': [1, 1], 'GSM1900699': [1, 1], 'GSM1900700': [1, 1], 'GSM1900701': [1, 1], 'GSM1900702': [0, 0], 'GSM1900703': [1, 1], 'GSM1900704': [1, 1], 'GSM1900705': [1, 1], 'GSM1900706': [1, 1], 'GSM1900707': [1, 0], 'GSM1900708': [1, 0], 'GSM1900709': [1, 1], 'GSM1900710': [1, 1], 'GSM1900711': [1, 1], 'GSM1900712': [1, 1], 'GSM1900713': [1, 1], 'GSM1900714': [1, 1], 'GSM1900715': [1, 1], 'GSM1900716': [1, 0], 'GSM1900717': [1, 0], 'GSM19007

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492', 'A_19_P00315493',
       'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4', 'BPI fold containing family A, member 3'], 'UNIGENE_ID': [nan, nan, 'Hs.642618', 'Hs.348522', 'Hs.360989'], 'ENSEMBL_ID': [nan, nan, 'ENST00000014930', 'ENST00000281830', 'ENST00000375454'], 'ACCESSION_STRING': [nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788', 'ref|NM_178466|ens|ENST00000375454|ens|ENST00000471233|tc

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the identifier and gene symbol keys
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to convert probe IDs to gene symbols
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Kidney_stones/gene_data/GSE73680.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Kidney_stones')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE73680', './preprocessed/Kidney_stones/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Kidney_stones/GSE73680.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Kidney_stones', the least common label is '0.0' with 6 occurrences. This represents 9.68% of the dataset.
The distribution of the feature 'Kidney_stones' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 22 occurrences. This represents 35.48% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

